In [181]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

#### Specifying the paths

In [182]:
csv_path = Path(
    r"\\wsl.localhost\Ubuntu-24.04\home\user\OF\attempt_2\results\csv\results.csv"
)
figures_path = Path(
    r"\\wsl.localhost\Ubuntu-24.04\home\user\OF\attempt_2\results/figures"
)
figures_path.mkdir(exist_ok=True, parents=True)

#### Loading in data

In [183]:
force_coeff_df = pd.read_csv(csv_path, parse_dates=["datetime"])
force_coeff_df["cl_cd"] = force_coeff_df["cl"] / force_coeff_df["cd"]
force_coeff_df["cl_cd_abs"] = (force_coeff_df["cl"] / force_coeff_df["cd"]).abs()
force_coeff_df["failed"] = (
    ~force_coeff_df[["no_clipping", "block_mesh", "check_mesh", "simple"]]
).any(axis=1)

df_filtered = force_coeff_df

df_filtered = force_coeff_df.dropna(subset=["cl", "cd"]).copy()
df_filtered = df_filtered[
    df_filtered["run_name"] == "5_degree_AoA_fixed_nu_tilda_reduced_yplus"
]
# df_filtered = df_filtered[
#     df_filtered["cl_cd_abs"] < 200
# ]
df_filtered["moving_avg"] = df_filtered["cl_cd_abs"].rolling(window=30).mean()

In [189]:
df_filtered.sort_values(by="cl_cd_abs", ascending=False)

,datetime,UUID,run_name,x0,x1,x2,x3,x4,x5,no_clipping,...,simple,vx,vy,vz,cl,cd,cl_cd,cl_cd_abs,failed,moving_avg
1453,2024-12-30 17:16:40.324946,0ee6b1bd-af39-49e6-a205-e4c9e3db9a51,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.311022,-0.388179,-0.576057,0.681258,0.105946,0.761899,True,...,True,99.61947,8.715574,0.0,0.197595,-0.000191,-1036.351421,1036.351421,False,55.664265
1187,2024-12-30 16:00:24.837342,e8f3c101-8a77-4d1f-aa09-31031a725ff2,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.600967,-0.577398,-0.737877,0.380478,0.386513,0.455492,True,...,True,99.61947,8.715574,0.0,0.216188,-0.000236,-914.412195,914.412195,False,NaN
1812,2024-12-30 19:56:23.895827,b308dc0e-d5a9-4a83-82c6-5ce1cc73aba0,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.319968,0.093075,-0.650005,0.785769,0.168778,1.405479,True,...,True,99.61947,8.715574,0.0,0.334500,-0.000400,-835.662362,835.662362,False,66.920178
1231,2024-12-30 16:08:25.356847,10cbd4c7-ed8a-40d6-8c84-3a55d3cbc284,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.735379,-0.521037,-0.260604,0.744460,0.271148,0.789393,True,...,True,99.61947,8.715574,0.0,0.259379,-0.000433,-599.695873,599.695873,False,NaN
1225,2024-12-30 16:06:30.023266,afa939e2-275f-481c-9683-1915ccdcf222,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.628930,1.206741,-0.724991,0.324524,0.953366,0.416324,True,...,True,99.61947,8.715574,0.0,0.547905,0.001419,386.210859,386.210859,False,NaN
1706,2024-12-30 18:59:35.388053,7bccea98-a3b8-4840-981c-b941a45220d6,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.392735,-0.893262,-0.161846,0.708337,-0.113752,0.512949,True,...,True,99.61947,8.715574,0.0,0.195689,0.000773,252.992810,252.992810,False,25.953121
1021,2024-12-30 15:34:46.203390,a6b0bc46-ead8-45d8-bf7c-31b864d591d4,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.658909,0.422954,-0.408254,0.290479,1.096998,-0.027268,True,...,True,99.61947,8.715574,0.0,0.543609,0.003156,172.231359,172.231359,False,NaN
1129,2024-12-30 15:48:21.269138,5dc0ec97-ed0a-4a0c-aca6-fc212999c974,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.412679,0.443798,-0.375875,0.702192,1.002144,0.944155,True,...,True,99.61947,8.715574,0.0,0.775737,-0.005101,-152.080544,152.080544,False,NaN
1165,2024-12-30 15:53:03.711527,811f3ce0-229c-49d0-9cc9-38b9521c5949,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.195443,0.565247,-0.226724,0.136192,1.254823,0.900455,True,...,True,99.61947,8.715574,0.0,0.846729,0.005976,141.686328,141.686328,False,NaN
1799,2024-12-30 19:51:06.310511,2b0c30d6-a1f8-4cbd-b4a7-0dcbfaf2c2ab,5_degree_AoA_fixed_nu_tilda_reduced_yplus,-0.542201,-1.220115,-0.390735,0.826111,0.169832,0.804890,True,...,True,99.61947,8.715574,0.0,0.541225,-0.003932,-137.633278,137.633278,False,37.855323


#### Progress plot

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(
    df_filtered["datetime"],
    df_filtered["cl_cd_abs"],
    label="Lift-drag ratio (Cl/Cd)",
    marker="o",
    color="tab:blue",
    alpha=0.25,
)
plt.plot(
    df_filtered["datetime"],
    df_filtered["cl_cd_abs"].cummax(),
    label="Cumulative max Cl/Cd",
    linestyle="--",
    color="tab:green",
)
plt.scatter(
    force_coeff_df[force_coeff_df["failed"]]["datetime"],
    -force_coeff_df[force_coeff_df["failed"]]["failed"],
    s=10,
    color="tab:red",
    alpha=0.1,
    label="Failed runs",
)

plt.plot(
    df_filtered["datetime"],
    df_filtered["moving_avg"],
    label="Moving average",
    linestyle="-.",
    color="tab:orange",
)

plt.title(f"Lift-drag ratio over time - {len(force_coeff_df)} simulations")
plt.xlabel("Time")
plt.ylabel("$C_l/C_d$ ratio")

# Use a locator to reduce the density of x-ticks
# locator = mdates.HourLocator(interval=1)
# formatter = mdates.DateFormatter("%H:%M")
# plt.gca().xaxis.set_major_locator(locator)
# plt.gca().xaxis.set_major_formatter(formatter)

plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(
    "results/figures/27122024 - OpenFOAM - Lift-drag ratio over time.png", dpi=300
)
plt.show()

#### Attempts at model reduction

In [ ]:
df_subset = df_filtered[
    (df_filtered["cl_cd_abs"] > 5.0) & (df_filtered["cl_cd_abs"] < 1005.0)
]

X = df_subset[["x0", "x1", "x2", "x3", "x4", "x5"]]
y = df_subset["cl_cd"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
percentage_error = np.mean(abs((y_test - y_pred) / y_test)) * 100

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Percentage Error: {percentage_error:.2f}%")

# Dummy method: Mean baseline
y_dummy_pred = [y_train.mean()] * len(y_test)
mse_dummy = mean_squared_error(y_test, y_dummy_pred)
mae_dummy = mean_absolute_error(y_test, y_dummy_pred)
percentage_error_dummy = np.mean(abs((y_test - y_dummy_pred) / y_test)) * 100

print("\nDummy Model Evaluation:")
print(f"Mean Squared Error (Dummy): {mse_dummy}")
print(f"Mean Absolute Error (Dummy): {mae_dummy}")
print(f"Mean Percentage Error (Dummy): {percentage_error_dummy:.2f}%")

plt.hist((y_test - y_pred) / y_test * 100, bins=100)
plt.xlabel("Error (%)")
plt.ylabel("Bin count")
plt.title("Model reduction with random forest")
plt.grid()

plt.savefig(
    "results/figures/27122024 - OpenFOAM - random forest model reduction.png", dpi=300
)

In [ ]:
pd.set_option("display.max_rows", None)
model_reduction_df = pd.DataFrame({"pred": y_pred, "test": y_test.values}).sort_values(
    by="test", ascending=False
)
model_reduction_df["error"] = model_reduction_df["test"] - model_reduction_df["pred"]
model_reduction_df["percent_error"] = (
    model_reduction_df["test"] - model_reduction_df["pred"]
) / model_reduction_df["test"]

model_reduction_df

In [ ]:
force_coeff_df = pd.read_csv(
    r"Z:\home\user\OF\attempt_2\custom_runs\cl_cd_1036.351_0ffbd6d2-2507-45ce-931a-2dc5ce4a3077\postProcessing\forceCoeffs\0\coefficient.dat",
    skiprows=12,
    sep="\t",
)
force_coeff_df.columns = [column_name.strip() for column_name in force_coeff_df.columns]

In [ ]:
fig, ax1 = plt.subplots()

# Plot Cl and Cd on the first y-axis
# (line1,) = ax1.plot(
#     force_coeff_df["# Time"], force_coeff_df["Cl"], label="$C_l$", color="tab:blue"
# )
(line2,) = ax1.plot(
    force_coeff_df["# Time"], force_coeff_df["Cd"], label="$C_d$", color="tab:green"
)
ax1.set_xlabel("Time")
ax1.set_ylabel("$C_l$, $C_d$")
ax1.set_ylim([-0.0005, 0.0005])

# Create a second y-axis for Cl/Cd
# ax2 = ax1.twinx()
# (line3,) = ax2.plot(
#     force_coeff_df["# Time"],
#     force_coeff_df["Cl"] / force_coeff_df["Cd"],
#     label="$C_l/C_d$",
#     color="tab:red",
# )
# ax2.set_ylabel("$C_l/C_d$")
# ax2.set_ylim([-2500, 100])

# Combine legends
# lines = [line1, line2, line3]
# labels = [line.get_label() for line in lines]
# ax1.legend(lines, labels, loc="upper right")

plt.title("Forces over time")
plt.tight_layout()
plt.savefig(
    "results/figures/30122024 - OpenFOAM - oscillatory solution to lift-drag ratio - zoomed in drag.png",
    dpi=300,
)
